<a href="https://colab.research.google.com/github/whitehatjr1001/Football-analysis/blob/main/FootBallpredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import warnings 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
warnings.filterwarnings('ignore')
url = "https://www.betexplorer.com/soccer/england/premier-league/results/"
soup = BeautifulSoup(requests.get(url).content,"html.parser")

In [7]:
def get_odd_or_text(td):
    if "data-odd" in td.attrs:
        return td["data-odd"]

    odd = td.select_one("[data-odd]")
    if odd:
        return odd["data-odd"]

    return td.get_text(strip=True)


In [8]:
all_data = []
for row in soup.select(".table-main tr:has(td)"):
    tds = [get_odd_or_text(td) for td in row.select("td")]
    round_ = row.find_previous("th").find_previous("tr").th.text
    all_data.append([round_, *tds])

df = pd.DataFrame(
    all_data, columns=["Round", "Match", "Score", "1", "X", "2", "Date"])


df['Home'] = [i.split('-')[0] for i in df['Match']]
df['Away'] = [i.split('-')[1] for i in df['Match']]

In [28]:
df.shape

(278, 14)

In [29]:
#reverse df
df = df.iloc[::-1]


cols = ['1','X','2']

df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df['HomeWin_Prob'] = round(1/df['1'],2)
df['DrawWin_Prob'] = round(1/df['X'],2)
df['AwayWin_Prob'] = round(1/df['2'],2)

df['HomeGoals'] = [i.split(':', 1)[0] for i in df['Score']]



In [30]:
df['AwayGoals'] = [i.split(':', 1)[1] if (':' in i and len(i.split(':', 1)) > 1) else '' for i in df['Score']]

In [31]:


cols = ['1','X','2']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df['HomeWin_Prob'] = round(1/df['1'],2)
df['DrawWin_Prob'] = round(1/df['X'],2)
df['AwayWin_Prob'] = round(1/df['2'],2)

df['HomeGoals'] = [i.split(':', 1)[0] for i in df['Score']]
df['AwayGoals'] = [i.split(':', 1)[1] if (':' in i and len(i.split(':', 1)) > 1) else '' for i in df['Score']]

In [33]:
def result(df):
  if df['HomeGoals']>df['AwayGoals']:
    return 1
  if df['HomeGoals']==df['AwayGoals']:
    return 0
  if df['HomeGoals']<df['AwayGoals']:
    return 2
  





In [35]:
df['Result'] = df.apply(result,axis=1)

In [36]:
df

,Round,Match,Score,1,X,2,Date,Home,Away,HomeWin_Prob,DrawWin_Prob,AwayWin_Prob,HomeGoals,AwayGoals,Result
0,29. Round,Arsenal-Leeds,4:1,1.26,6.33,10.91,Yesterday,Arsenal,Leeds,0.79,0.16,0.09,4,1,1
1,29. Round,Bournemouth-Fulham,2:1,2.56,3.26,2.89,Yesterday,Bournemouth,Fulham,0.39,0.31,0.35,2,1,1
2,29. Round,Brighton-Brentford,3:3,1.61,4.12,5.57,Yesterday,Brighton,Brentford,0.62,0.24,0.18,3,3,0
3,29. Round,Chelsea-Aston Villa,0:2,1.62,3.93,5.87,Yesterday,Chelsea,Aston Villa,0.62,0.25,0.17,0,2,2
4,29. Round,Crystal Palace-Leicester,2:1,2.39,3.29,3.13,Yesterday,Crystal Palace,Leicester,0.42,0.30,0.32,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,1. Round,Fulham-Liverpool,2:2,10.45,6.10,1.28,06.08.2022,Fulham,Liverpool,0.10,0.16,0.78,2,2,0
274,1. Round,Leeds-Wolves,2:1,2.43,3.34,3.00,06.08.2022,Leeds,Wolves,0.41,0.30,0.33,2,1,1
275,1. Round,Newcastle-Nottingham,2:0,1.58,4.04,6.19,06.08.2022,Newcastle,Nottingham,0.63,0.25,0.16,2,0,1
276,1. Round,Tottenham-Southampton,4:1,1.36,5.20,8.69,06.08.2022,Tottenham,Southampton,0.74,0.19,0.12,4,1,1


In [ ]:
df.drop(['Round','Score','Date','1','X','2','Match','HomeGoals','AwayGoals'],axis=1,inplace=True)


In [88]:
df.drop(44)
df.fillna(0) 


,Home,Away,HomeWin_Prob,DrawWin_Prob,AwayWin_Prob,Result
0,13,18,0.79,0.16,0.09,1
1,14,2,0.39,0.31,0.35,1
2,19,7,0.62,0.24,0.18,0
3,10,6,0.62,0.25,0.17,2
4,1,11,0.42,0.30,0.32,1
...,...,...,...,...,...,...
245,3,14,0.88,0.11,0.05,1
246,12,1,0.84,0.13,0.07,1
247,6,4,0.28,0.26,0.51,2
248,4,3,0.22,0.23,0.59,1


In [83]:

df.fillna(method='ffill') # forward fill
df.fillna(method='bfill') # backward fill


,Home,Away,HomeWin_Prob,DrawWin_Prob,AwayWin_Prob,Result
0,13,18,0.79,0.16,0.09,1
1,14,2,0.39,0.31,0.35,1
2,19,7,0.62,0.24,0.18,0
3,10,6,0.62,0.25,0.17,2
4,1,11,0.42,0.30,0.32,1
...,...,...,...,...,...,...
245,3,14,0.88,0.11,0.05,1
246,12,1,0.84,0.13,0.07,1
247,6,4,0.28,0.26,0.51,2
248,4,3,0.22,0.23,0.59,1


In [89]:
df.isna().sum()

Home            0
Away            0
HomeWin_Prob    1
DrawWin_Prob    1
AwayWin_Prob    1
Result          0
dtype: int64

In [41]:
hold_out = df[250::]
hold_out.drop(['Result'],axis=1,inplace=True)
hold_out = hold_out.reset_index(drop=True)
df = df[0:250]

In [55]:
model_recode = {'Southampton':0,
              'Crystal Palace':1,
              'Fulham':2,
              'Liverpool':3,
              'Manchester Utd':4,
              'Newcastle':5,
              'Aston Villa':6,
              'Brentford':7,
              'Tottenham':8,
              'West Ham':9,
              'Chelsea':10,
              'Leicester':11,
              'Manchester City':12,
              'Arsenal':13,
              'Bournemouth':14,
              'Everton':15,
              'Wolves':16,
              'Nottingham':17,
              'Leeds':18,
              'Brighton':19}


ValueError: ignored

,Home,Away,HomeWin_Prob,DrawWin_Prob,AwayWin_Prob
0,13.0,18.0,0.79,0.16,0.09
1,14.0,2.0,0.39,0.31,0.35
2,19.0,7.0,0.62,0.24,0.18
3,10.0,6.0,0.62,0.25,0.17
4,1.0,11.0,0.42,0.30,0.32
...,...,...,...,...,...
245,3.0,14.0,0.88,0.11,0.05
246,12.0,1.0,0.84,0.13,0.07
247,6.0,4.0,0.28,0.26,0.51
248,4.0,3.0,0.22,0.23,0.59


Home            0
Away            0
HomeWin_Prob    1
DrawWin_Prob    1
AwayWin_Prob    1
dtype: int64

In [90]:
# iterate over columns
for key, value in df['Home'].iteritems():
    df['Home'] = df['Home'].apply(lambda x: model_recode.get(x,x))

for key, value in df['Away'].iteritems():
    df['Away'] = df['Away'].apply(lambda x: model_recode.get(x,x))

for key, value in hold_out['Home'].iteritems():
    hold_out['Home'] = hold_out['Home'].apply(lambda x: model_recode.get(x,x))

for key, value in hold_out['Away'].iteritems():
    hold_out['Away'] = hold_out['Away'].apply(lambda x: model_recode.get(x,x))

X = df.drop('Result',axis=1)
y = df['Result']

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [92]:
def model_tuning_GS(model, parameter_dict):
    """Function to perform hyperparameter turning for the classification models using GridSearch."""
    # inspect the model params.
    model.get_params()
    # define the parameters using a dictionary that we want to test.
    model_grid = parameter_dict
    # initialise a GSCV object with the model as an argument. scoring is set to accuracy and CV set to 10.
    Grid_model = GridSearchCV(estimator=model, param_grid=model_grid, cv=10, scoring="accuracy")
    # fit the model to data.
    Grid_model.fit(X_train, y_train)
    # extract the best estimator, accuracy score and print them.
    print("GridSearchCV results:", model.__class__.__name__)
    # print best estimator
    print("Best Estimator:\n", Grid_model.best_estimator_)
    # printing the mean cross-validated score of the best_estimator:
    print("\n Best Score:\n", Grid_model.best_score_)
    # printing the parameter setting that gave the best results on the hold out testing data.:
    print("\n Best Hyperparameters:\n", Grid_model.best_params_)

In [93]:
parameter_dict = {'n_estimators': [1, 2, 3, 4, 5],
                  'max_depth': [1, 2, 3, 5, 8],
                  'min_samples_leaf': [0.1, 0.2],
                  'criterion':['gini', 'entropy', 'log_loss']}

model_tuning_GS(RandomForestClassifier(random_state=42), parameter_dict)

ValueError: ignored